In [3]:
import transformers
import torch, math, einops
from src import Tokenizer, linear_unidirectional_graph_maker

In [4]:
tokenizer = Tokenizer('gpt2')
pretraied = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [7]:
pretraied

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [3]:
from src.GPT2 import GPT2, GPT2_Encoder, GPT2_LM_Head

encoder=GPT2_Encoder()
decoder=GPT2_LM_Head()

model=GPT2(encoder, decoder, tokenizer,dropout=0)

model.load_from_original(pretraied)

AttributeError: 'GPT2Model' object has no attribute 'lm_f'

In [4]:
def split_heads(tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

In [5]:
x=torch.randint(0, 50257, (1, 10))
x=torch.randn([1,13,768])

sequence_lenght=17
batch_size=1
n_heads=13
d_Embedding=64*n_heads
Q=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])
K=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])
V=torch.randn([1,n_heads,sequence_lenght,d_Embedding//n_heads])


out, att = pretraied.transformer.h[0].attn._attn(Q,K,V,None,None)
#out=out.permute(1, 0, 2).contiguous().view(batch_size, sequence_lenght, d_Embedding)
out.shape,att.shape
out=out.permute(0,2,1,3).view(sequence_lenght,n_heads,d_Embedding//n_heads)



In [6]:
att_pretrained = einops.einsum(Q,K,'... s e, ... t e -> ... s t')/math.sqrt(K.size(-1))
alignments=att_pretrained
query_length, key_length = Q.size(-2), K.size(-2)

causal_mask = pretraied.transformer.h[0].attn.bias[:, :, key_length - query_length : key_length, :key_length].to(torch.bool)
mask_value = -torch.tensor(float('inf'))
att_pretrained = torch.where(causal_mask, att_pretrained, mask_value)
alignments=att_pretrained
att_pretrained = torch.nn.functional.softmax(att_pretrained, dim=-1)
att_pretrained = att_pretrained.view(n_heads,sequence_lenght,sequence_lenght)

(att_pretrained==att).all()



tensor(True)

In [7]:
from src.transformerMP import attention_message

graph_maker=linear_unidirectional_graph_maker(40)

edge_index=graph_maker(sequence_lenght)
senders, receivers = edge_index

Q1=Q.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)
K1=K.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)
V1=V.view(n_heads,sequence_lenght,d_Embedding//n_heads).permute(1,0,2)

#o,a=attention_message(Q,K,V,edge_index,0)
#o.view(sequence_lenght,d_Embedding)

N=20

i,j=receivers[N],senders[N]
print((Q1[receivers]*K1[senders]).sum(dim=-1)[N]/math.sqrt(K.size(-1)))

print(alignments.view(n_heads,sequence_lenght,sequence_lenght)[:,i,j])

tensor([ 0.9323,  1.3134,  0.0814, -0.7573,  1.5721,  1.6462,  0.7974, -0.1905,
        -0.8649, -1.7122, -0.2299,  0.9718, -0.9527])
tensor([ 0.9323,  1.3134,  0.0814, -0.7573,  1.5721,  1.6462,  0.7974, -0.1905,
        -0.8649, -1.7122, -0.2299,  0.9718, -0.9527])


In [8]:
from src.transformerMP import normalize_strength
from math import sqrt
def attention_message(Q:torch.Tensor,
                      K:torch.Tensor,
                      V:torch.Tensor,
                      edge_index:torch.Tensor,
                      att_dropout=0.1
                      ):

    senders, receivers = edge_index
    N,h,d=K.shape   

    #Q.K^T 
    att=(Q[receivers]*K[senders]).sum(dim=-1)/sqrt(d)

    #softmax    
    att = torch.exp(att+3-att.max()) #could be done in-plase using the function att.exp_() if memory is a bootleneck
    attention = normalize_strength(att, receivers, N, h)

    #Dropout
    #att=attention_dropout(attention, att_dropout)

    #softmax*V
    att = einops.einsum(attention,V[senders],' ... , ... c -> ... c')
    out = torch.zeros_like(V,device=V.device)

    return out.index_add(0,receivers,att), attention #could be done in-place using the function out.index_add_()

In [9]:
out_function,a=attention_message(Q1,K1,V1,edge_index,0)

i=3
print(edge_index[:,receivers==j])
print(att_pretrained[i,j:,j])
print(a[senders==j,i])
asd=True
for j in range(att_pretrained.shape[-1]):
    asd= asd and torch.allclose(att_pretrained[:,j:,j],a.permute(1,0)[:,senders==j],1e-4,1e-4)
asd
    

tensor([[0, 1, 2, 3, 4, 5],
        [5, 5, 5, 5, 5, 5]])
tensor([0.0936, 0.1358, 0.0410, 0.2424, 0.2716, 0.0526, 0.2126, 0.0540, 0.1150,
        0.0570, 0.0814, 0.0564])
tensor([0.0936, 0.1358, 0.0410, 0.2424, 0.2716, 0.0526, 0.2126, 0.0540, 0.1150,
        0.0570, 0.0814, 0.0564])


True

In [10]:
V=V.view(n_heads,sequence_lenght,d_Embedding//n_heads)
att_pretrained.shape,V.shape
print(att_pretrained.shape,V.shape)
AV_pretrained=einops.einsum(att_pretrained,V,'h i j, h j e -> h i j e')
print(AV_pretrained.shape)

if not (AV_pretrained.sum(-2)==torch.matmul(att_pretrained,V)).all(): #true
    print (False)

print(a.shape,V1[senders].shape)
dsa=einops.einsum(a,V1[senders],'..., ... e -> ... e')
dsa.shape==V1[senders].shape==(152, n_heads, d_Embedding//n_heads) #True

N=9
i,j=senders[N],receivers[N]
print(torch.allclose(dsa[N],AV_pretrained[:,j,i],1e-3,1e-3))

torch.Size([13, 17, 17]) torch.Size([13, 17, 64])
torch.Size([13, 17, 17, 64])
torch.Size([153, 13]) torch.Size([153, 13, 64])
True


In [11]:
out_pretrained=AV_pretrained.sum(-2).permute(1,0,2)
dsum=torch.zeros_like(V1).index_add(0,receivers,dsa)

print(out_pretrained.shape,dsum.shape)
print(torch.allclose(out_pretrained,dsum,1e-3,1e-3))
torch.allclose(out,out_function,1e-3,1e-3)
#print((dsa[N]==asd[:,j,i]))

torch.Size([17, 13, 64]) torch.Size([17, 13, 64])
True


True

# aasdasda

In [12]:
sequence_length=13
heads=12
d_Embedding=64*heads

In [13]:
def split_heads(tensor, num_heads, attn_head_size):
        """
        Splits hidden_size dim into attn_head_size and num_heads
        """
        new_shape = tensor.size()[:-1] + (num_heads, attn_head_size)
        tensor = tensor.view(new_shape)
        return tensor.permute(0, 2, 1, 3)  # (batch, head, seq_length, head_features)

def merge_heads(tensor, num_heads, attn_head_size):
    """
    Merges attn_head_size dim and num_attn_heads dim into hidden_size
    """
    tensor = tensor.permute(0, 2, 1, 3).contiguous()
    new_shape = tensor.size()[:-2] + (num_heads * attn_head_size,)
    return tensor.view(new_shape)

def original_c_attn(x,i=0):
        query, key, value = pretraied.transformer.h[i].attn.c_attn(x).split(d_Embedding, dim=2)

        dim=d_Embedding//heads
        query = split_heads(query, heads, dim)
        key = split_heads(key, heads, dim)
        value = split_heads(value, heads, dim)

        return query,key,value

In [14]:

x=torch.randn([1,sequence_length,d_Embedding])

edge_index=graph_maker(13)
Qp,Kp,Vp=original_c_attn(x)
Q,K,P=model.transformer_blocks[0].attention_block.make_QKV(x)

In [15]:
Qp.shape

torch.Size([1, 12, 13, 64])

In [16]:
QQ=Qp.view([heads,sequence_length,d_Embedding//heads]).permute(1,0,2)

QQ[:,0,0],Q[:,0,0]



(tensor([ 2.5202,  2.1009,  5.4433, -4.5101,  3.7691,  3.1631,  8.9820, -3.0704,
         -1.4016, -2.7364,  2.7083, -4.1566, 12.2631],
        grad_fn=<SelectBackward0>),
 tensor([ 2.5202,  2.1009,  5.4433, -4.5101,  3.7691,  3.1631,  8.9820, -3.0704,
         -1.4016, -2.7364,  2.7083, -4.1566, 12.2631],
        grad_fn=<SelectBackward0>))

# sawrar

In [24]:
from torch import nn
ln_1=nn.LayerNorm(d_Embedding,eps=1e-3)
ln_2=nn.LayerNorm(d_Embedding,eps=1e-3)

def forward(hidden_states):

    residual = hidden_states  # residual

    hidden_states = ln_1(hidden_states)  # normalization

    attn_outputs = pretraied.transformer.h[0].attn(hidden_states)
    attn_output = attn_outputs[0]  # output_attn: a, present, (attentions)
    outputs = attn_outputs[1:]
    # residual connection
    hidden_states = attn_output + residual  # attention



    residual = hidden_states  # reisdual again

    hidden_states = ln_2(hidden_states)  # normalization

    feed_forward_hidden_states = pretraied.transformer.h[0].mlp(hidden_states)  # mlp
    # residual connection
    hidden_states = residual + feed_forward_hidden_states

    outputs = (hidden_states,) + outputs[1:]

    #in my model it only return hidden states
    # hidden_states, present, (attentions, cross_attentions)
    return outputs


In [26]:
x=torch.randn((1,sequence_length,d_Embedding))
out1=forward(x)
out2=pretraied.transformer.h[0](x)

In [47]:
out1[0][0],out2[0][0]

(tensor([[ 11.0140,   1.3385,   0.3527,  ...,   0.4152,  -6.2672,  -1.3940],
         [  6.7424, -11.9847,  16.5153,  ...,   1.1466,  -2.2571,  -1.7462],
         [ 14.0308,   6.2891,   1.8980,  ...,   1.1527,   2.8099,   0.7151],
         ...,
         [  3.4113,  10.9683,   9.1666,  ...,  18.5762,   3.4282,  15.2770],
         [ 16.7814,  -7.9987, -23.0774,  ...,   1.2276,  -1.3529,   8.3471],
         [ 20.9909,  -1.9449,  -4.7618,  ...,   8.8635,  -3.1906,   4.2272]],
        grad_fn=<SelectBackward0>),
 tensor([[  8.6352,   3.6732,  -4.3042,  ...,  17.0297,   5.4470,   0.7072],
         [ 32.1582,  -6.3796,  -7.5093,  ...,  12.4453,  -4.0244,  -7.5346],
         [ 31.5250,  -5.3637,  -8.9045,  ...,   9.0292,   0.0418,  -0.5986],
         ...,
         [  7.7772,   6.9585,  -0.8895,  ...,   8.9632,   8.2517,  26.1103],
         [ 11.0655,  -7.6909, -18.9837,  ...,   0.9446,   4.0766,  19.9787],
         [ 18.9379,   1.8972, -19.4432,  ...,   5.3744,  12.9888, -10.2606]],
        gr

In [8]:
x = tokenizer("Hello world! good i saw you")
graph_maker=linear_unidirectional_graph_maker(40)

edge_index=graph_maker(x.shape[0])
tokenizer.decode(pretraied(x)[0].argmax(dim=-1)),tokenizer.decode(model(x,edge_index).argmax(dim=-1))


(",,\n morning'm you guys", 'ㅋ DW YORKettel Democracy YORK ACA')